In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-1-e4476ecac24a>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
############################################
################# Load data ################
############################################

#
def get_sessions_from_directory_listing(root_dir,
                                       animal_id):
    
    sessions = os.listdir(root_dir+animal_id)
    print ("sessions: ", sessions)
    
    return sessions
    
#

animal_ids = [
    #'DON-009191',
    'DON-009192',
    #'DON-010473',
    #'DON-010477'
]
              

root_dir = '/media/cat/4TB/donato/'
animal_id = animal_ids[0]
dir_ = '/002P-F/tif/'


session = '20220310'
#sessions = get_sessions_from_directory_listing(root_dir,
#                                              animal_id)

print ("PRocessing: ", animal_id)
#
#
print ("processing: ", session)
c = calcium.Calcium()
c.root_dir = root_dir
c.data_dir = os.path.join(root_dir, animal_id, session+dir_, 'suite2p','plane0')
c.animal_id = animal_id
c.session = session
c.detrend_model_order = 1

#
c.load_suite2p()
c.save_python = True
c.save_matlab = False

# 
c.load_binarization()
binarization_method = 'upphase'
if binarization_method=='onphase':
    traces = c.F_onphase_bin
elif binarization_method=='upphase':
    traces = c.F_upphase_bin
else:
    print ("METHOD NOT FOUND")

# print ("binarized data: ", traces.shape)

###################################################################
########## cleanup cells + compute pairwise correlations ##########
###################################################################
c.load_footprints()
c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
c.corr_max_percent_overlap = 0.25       # max overlap for overlap method
c.corr_threshold = 0.3                  # max correlation allowed for high overlap

#  
c.corr_delete_method = 'lowest_snr' #'highest_connected', lowest_snr'
c.recompute_deduplication = False

#
c.correlation_datatype = 'filtered'      # filtered vs. upphase

#
c.compute_correlations()

#
c.remove_duplicate_neurons()     # this removes duplicate neurons and saves non-duplicate version of correlation array

#
print ("DONE")


PRocessing:  DON-009192
processing:  20220310
pre clean corr array:  (778, 778, 2)
clean cell ids:  (612,)
pre clean corr array:  (612, 612)
DONE


(778, 111598)


In [29]:
from tqdm import tqdm, trange


corrs = np.zeros((F_filtered.shape[0], F_filtered.shape[0], 2))
corrs_clipped = np.zeros((F_filtered.shape[0], F_filtered.shape[0], 2))

#
for k in trange(F_filtered.shape[0]): #,desc='computing intercell correlation'):
    temp1 = F_filtered[k][::5]
    temp1 = temp1 - np.median(temp1)
    #
    for p in range(k, F_filtered.shape[0],1):
        temp2 = F_filtered[p][::5]
        temp2 = temp2 - np.median(temp2)
        corr = scipy.stats.pearsonr(temp1,
                                    temp2)
        corrs[k, p, 0] = corr[0]
        corrs[k, p, 1] = corr[1]
        
        # corr = scipy.stats.pearsonr(temp1[300:],
        #                             temp2[300:])
        # corrs_clipped[k, p, 0] = corr[0]
        # corrs_clipped[k, p, 1] = corr[1]

100%|██████████| 778/778 [01:07<00:00, 11.49it/s] 


In [30]:

print (corrs.shape)

plt.figure()
ax=plt.subplot(121)
plt.imshow(corrs[:,:,0])
plt.colorbar()
ax=plt.subplot(122)
plt.title("clipped correlations")
plt.imshow(corrs_clipped[:,:,0])
plt.colorbar()

plt.show()

(778, 778, 2)


In [31]:
from scipy.signal import savgol_filter

from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt


def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


#
plt.figure()
t = np.arange(F_filtered[0].shape[0])/30./60.
for k in range(10):
    plt.plot(t,F_filtered[k]+k*1000)
    
    y = F_filtered[k]
    y = y - np.median(y)
    
    plt.plot(t, y +k*1000)
    
    
plt.xlabel("Time (mins)")
plt.ylabel("Filtered [ca] time series")
plt.show()



In [30]:
d1 = np.load('/media/cat/4TB1/donato/DON-009191/20220305/002P-F/tif/suite2p/plane0/allcell_correlation_array_filtered.npy')
print (d1.shape)
d2 = np.load('/media/cat/4TB1/donato/DON-009191/20220317/002P-F/tif/suite2p/plane0/goodcell_correlations_array_post_deduplication_filtered.npy')
print (d2.shape)

plt.figure()
plt.imshow(d2)
plt.colorbar()
plt.show()

(237, 237, 2)
(437, 437)


In [36]:
from scipy.spatial import distance
from scipy.cluster import hierarchy
import seaborn as sns

row_linkage = hierarchy.linkage(
    distance.pdist(d2), method='average')

col_linkage = hierarchy.linkage(
    distance.pdist(d2.T), method='average')

cmap = sns.diverging_palette(h_neg=210, h_pos=350, s=90, l=30, as_cmap=True)

#
sns.clustermap(d2, 
               #row_linkage=row_linkage, 
               #col_linkage=col_linkage, 
               #row_colors=network_colors, 
               method="average",
               #col_colors=network_colors, 
               figsize=(13, 13), 
               cmap=cmap)



/home/cat/anaconda3/envs/manifolds/lib/python3.8/site-packages/seaborn/matrix.py:654: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)


In [2]:
################################################
################################################
################################################

session = '20220513'
data1 = np.load('/media/cat/4TB1/donato/DON-010477/20220513/002P-F/tif/suite2p/plane0/cell_correlations_filtered.npy')
data2 = np.load('/media/cat/4TB1/donato/DON-010477/20220513/002P-F/tif/suite2p/plane0/cell_correlations_upphase.npy')

print (data1.shape, data2.shape)

print (data1[1])

corrs1 = data1[:,2]
print (corrs1)
corrs2 = data2[:,2]
print (corrs2)


plt.figure()
plt.ylabel('Correlation value computed on filtered fluorescence')
plt.scatter(corrs1, corrs2,
           alpha=.1,
           edgecolor='black')


plt.xlabel('Correlation value computed on upphase binarized data')
plt.suptitle(animal_id + " " + session)
plt.show()
    

FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/4TB1/donato/DON-010477/20220513/002P-F/tif/suite2p/plane0/cell_correlations_filtered.npy'

In [4]:
d = np.load('/media/cat/4TB/donato/DON-010477/20220513/002P-F/tif/suite2p/plane0/goodcell_correlations_array_post_deduplication_filtered.npy')
print (d.shape)

plt.figure()
plt.imshow(d)
plt.colorbar()
plt.show()

(709, 709)


In [7]:
plt.figure()
plt.hist(d.flatten(), bins=np.arange(-1,1,0.1))
plt.show()